# 付録 10.4.5: Haikuをサブエージェントとして使用する

このレシピでは、Claude 3 Haikuサブエージェントモデルを使用して、Appleの2023年の財務収益報告書を分析し、収益発表PDFから関連情報を抽出する方法を示します。その後、Claude 3 Sonnetを使用して質問に対する応答を生成し、その応答に伴うグラフをmatplotlibを使用して作成します。

## ステップ 1: 環境を設定する
まず、必要なライブラリをインストールし、`Anthropic API`クライアントを設定しましょう。

In [ ]:
pip install -qUr requirements.txt

In [ ]:
# 必要なライブラリをインポート
import boto3
import fitz
from PIL import Image
import io
from concurrent.futures import ThreadPoolExecutor
import requests
import os

session = boto3.Session()
region = session.region_name

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

## ステップ2: ドキュメントを集めて質問する
この例では、2023年度のAppleのすべての財務諸表を使用し、年間の純売上について質問します。

In [ ]:
# Appleの収益発表PDFのURLリスト
pdf_urls = [
    "https://www.apple.com/newsroom/pdfs/fy2023-q4/FY23_Q4_Consolidated_Financial_Statements.pdf",
    "https://www.apple.com/newsroom/pdfs/fy2023-q3/FY23_Q3_Consolidated_Financial_Statements.pdf",
    "https://www.apple.com/newsroom/pdfs/FY23_Q2_Consolidated_Financial_Statements.pdf",
    "https://www.apple.com/newsroom/pdfs/FY23_Q1_Consolidated_Financial_Statements.pdf"
]

# ユーザーの質問
QUESTION = "How did Apple's net sales change quarter to quarter in the 2023 financial year and what were the key contributors to the changes?"

## ステップ3: PDFをダウンロードして画像に変換する  
次に、収益発表のPDFをダウンロードし、それをbase64エンコードされたPNG画像に変換する関数を定義します。これを行う必要があるのは、これらのPDFが従来のPDFパーサーでは解析が難しいテーブルでいっぱいだからです。画像に変換して、それをHaikuに渡す方が簡単です。

```download_pdf```関数は、指定されたURLからPDFファイルをダウンロードし、指定されたフォルダーに保存します。```pdf_to_pngs```関数は、PDFをPNG画像のリストに変換します。

In [ ]:
# URLからPDFファイルをダウンロードし、指定されたフォルダーに保存する関数
def download_pdf(url, folder):
    response = requests.get(url)
    if response.status_code == 200:
        file_name = os.path.join(folder, url.split("/")[-1])
        with open(file_name, "wb") as file:
            file.write(response.content)
        return file_name
    else:
        print(f"{url} からのPDFのダウンロードに失敗しました")
        return None
    
# PDFをbase64エンコードされたPNG画像のリストに変換する関数を定義
def pdf_to_png(pdf_path, quality=75, max_size=(1024, 1024)):
    # PDFファイルを開く
    doc = fitz.open(pdf_path)
    pdf_to_png_images = []

    # PDFの各ページを反復処理
    for page_num in range(doc.page_count):
        # ページを読み込む
        page = doc.load_page(page_num)

        # ページをPNG画像としてレンダリング
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

        # pixmapをPIL Imageに変換
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # 最大サイズを超える場合は画像をリサイズ
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)

        # PIL画像をバイトに変換
        image_data = io.BytesIO()
        image.save(image_data, format='PNG', optimize=True, quality=quality)
        image_data.seek(0)
        pdf_to_png_image = image_data.getvalue()

        # PNG画像のバイトをリストに追加
        pdf_to_png_images.append(pdf_to_png_image)

    # PDFドキュメントを閉じる
    doc.close()

    return pdf_to_png_images

# ダウンロードしたPDFを保存するフォルダー
folder = "./images/using_sub_agents"

# PDFを同時にダウンロード
with ThreadPoolExecutor() as executor:
    pdf_paths = list(executor.map(download_pdf, pdf_urls, [folder] * len(pdf_urls)))

# pdf_pathsからNoneの値（ダウンロード失敗）を削除
pdf_paths = [path for path in pdf_paths if path is not None]

`ThreadPoolExecutor`を使用してPDFを同時にダウンロードし、ファイルパスを`pdf_paths`に保存します。

## ステップ 4: ソネットを使用してハイクのための特定のプロンプトを生成する
オーパスをオーケストレーターとして使用し、ユーザーが提供した質問に基づいて各ハイクサブエージェントのために特定のプロンプトを書かせましょう。

In [ ]:
def generate_haiku_prompt(question):
    prompt = f"""Based on the following question, please generate a specific prompt for an LLM sub-agent to extract relevant information from an earning's report PDF. Each sub-agent only has access to a single quarter's earnings report. Output only the prompt and nothing else.\n\nQuestion: {question}"""
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": prompt }
            ]
        }
    ]

    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
    }

    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text']

haiku_prompt = generate_haiku_prompt(QUESTION)
print(haiku_prompt)

## ステップ5: PDFから情報を抽出する
さて、質問を定義し、サブエージェントのHaikuモデルを使用してPDFから情報を抽出しましょう。各モデルからの情報を整然と定義された一連のXMLタグにフォーマットします。

In [ ]:
def extract_info(pdf_path, haiku_prompt):
    pdf_pngs = pdf_to_png(pdf_path)

    messages = [
        {
            "role": "user",
            "content": [
                *[{"image": {"format": 'png', "source": {"bytes": pdf_png}}} for pdf_png in pdf_pngs],
                {"text": haiku_prompt}
            ]
        }
    ]

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": messages,
    }
    response = bedrock_client.converse(**converse_api_params)

    return response['output']['message']['content'][0]['text'], pdf_path

def process_pdf(pdf_path):
    return extract_info(pdf_path, QUESTION)

# Haikuサブエージェントモデルを使用してPDFを同時に処理する
with ThreadPoolExecutor() as executor:
    extracted_info_list = list(executor.map(process_pdf, pdf_paths))

extracted_info = ""
# 各モデル呼び出しから抽出された情報を表示する
for info in extracted_info_list:
    extracted_info += "<info quarter=\"" + info[1].split("/")[-1].split("_")[1] + "\">" + info[0] + "</info>\n"
print(extracted_info)

PDFから情報を同時に抽出するためにサブエージェントモデルを使用し、抽出した情報を統合します。その後、強力なモデルのためにメッセージを準備し、質問と抽出した情報を含めて、応答と`matplotlib`コードを生成するように依頼します。

## ステップ6: 情報をSonnetに渡して応答を生成する  
各PDFからサブエージェントを使用して情報を取得したので、実際に質問に答えるためにOpusを呼び出し、応答に添付するグラフを作成するコードを書きましょう。

In [ ]:
# メッセージを強力なモデルのために準備する
messages = [
    {
        "role": "user",
        "content": [
            {"text": f"以下のAppleの収益発表から抽出した情報に基づいて、質問に対する回答を提供してください: {QUESTION}\n\nまた、回答に伴うmatplotlibライブラリを使用したPythonコードを生成してください。コードは<code>タグで囲んでください。\n\n抽出された情報:\n{extracted_info}"}
        ]
    }
]

# 強力なモデルを使用してmatplotlibコードを生成する
converse_api_params = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",
    "messages": messages,
    "inferenceConfig": {"maxTokens": 4096},
}
response = bedrock_client.converse(**converse_api_params)

generated_response = response['output']['message']['content'][0]['text']
print("生成された応答:")
print(generated_response)

## ステップ 7: 応答を抽出し、Matplotlib コードを実行する
最後に、生成された応答から matplotlib コードを抽出し、収益成長トレンドを視覚化するために実行します。

```extract_code_and_response``` 関数を定義して、生成された応答から matplotlib コードと非コード応答を抽出します。非コード応答を印刷し、matplotlib コードが見つかった場合はそれを実行します。

モデルが書いたコードに対してサンドボックス外で ```exec``` を使用することは良いプラクティスではありませんが、このデモの目的のためにそれを行っています :)

In [ ]:
# Extract the matplotlib code from the response
# レスポンスからコードと非コード部分を抽出する関数
def extract_code_and_response(response):
    start_tag = "<code>"
    end_tag = "</code>"
    start_index = response.find(start_tag)
    end_index = response.find(end_tag)
    if start_index != -1 and end_index != -1:
        code = response[start_index + len(start_tag):end_index].strip()
        non_code_response = response[:start_index].strip()
        return code, non_code_response
    else:
        return None, response.strip()

matplotlib_code, non_code_response = extract_code_and_response(generated_response)

print(non_code_response)
if matplotlib_code:

    # 抽出したmatplotlibコードを実行する
    exec(matplotlib_code)
else:
    print("レスポンスにmatplotlibコードが見つかりませんでした。")